# Prediction 결과 업로드

## 개요
추천 모델로 Prediction한 결과를 실제 서비스에 적용하기 위해서 다양한 방법이 있을 수 있습니다. 그 중 한가지 방법은 MLS를 사용하여 추천 결과를 서빙하는 것입니다.  
MLS를 사용하기 위해서 Prediction한 결과를 S3에 업로드해야 합니다. 이 예제는 Prediction 결과를 MLS에 서빙할 수 있는 포맷으로 가공한 후 S3에 업로드하는 내용을 설명합니다.  
참고로, MLS에 대한 정보는 MLS 문서를 참고하시기 바랍니다.


## Prediction 데이터 가공

Prediction 파트에서 생성한 Prediction 데이터는 빅쿼리에 저장되어 있습니다.

In [1]:
from pydatafabric.gcp import import_bigquery_ipython_magic
import_bigquery_ipython_magic()

In [2]:
%%bq

SELECT *
FROM temp_1d.example_model_prediction_result

BigQuery execution took 4 seconds.


,svc_mgmt_num,prod_id,prod_nm,score
0,02077c121754329abf07f3caf7020bf21da24c66099f21...,62,open_mbr,0.483434
1,0108b6e0762edc413402983ece4ba019eb52717354aad0...,62,open_mbr,0.573481
2,01dafebfb858952a269d31d6a2e6c07a9b90c28c29d07f...,62,open_mbr,0.397717
3,0185176451c6a72beec8186d92fa71605dbef15cd95fbf...,62,open_mbr,0.584078
4,0148a2bf2ed821cff9c8d85dc05a5ed4c4d9d74edd7757...,62,open_mbr,0.340511
...,...,...,...,...
595,00f995124a7f7cc9c3cf67f4d21d19a602c0800d91beb8...,62,open_mbr,0.416590
596,021a2ea804395d344879af74033f75a2a0052acd1c2086...,62,open_mbr,0.416590
597,0148a2bf2ed821cff9c8d85dc05a5ed4c4d9d74edd7757...,140,global,0.743495
598,01cd81d8e0e1e6c3fd8f87bb6af0c2e66f9bba2a4269f4...,140,global,0.743495


우선 Prediction 결과 데이터를 메타 데이터와 조인하여 추천 서빙에 필요한 정보를 추가합니다. 참고로 여기서는 베이커리 혜택에 대한 추천 결과를 생성합니다.

In [3]:
from pydatafabric.gcp import bq_insert_overwrite

query = f"""
    SELECT ...
"""

bq_insert_overwrite(sql=query, destination="emart-datafabric.temp_1d.example_prediction_post_proc")

query: 
    SELECT b.svc_mgmt_num,
           b.benefit_id as prod_id,
           concat(b.brand_nm, ':::', b.benefit_nm) as prod_nm,
           'Y' as impression_yn,
           a.score,
           '1' as percent_rk,
           b.reco_type,
           'example_model' as model,
           PARSE_DATE('%Y-%m-%d', '2020-09-01') as dt 
    FROM temp_1d.example_model_prediction_result a
    join 
    (
        select * 
        ,      case concat(benefit_type, '_', ifnull(category_medium_name, '#'))
                                when 'VIP_PICK_영화' then 'mbr_url_movie' when 'CULTURE_공연/전시' then 'mbr_url_culture' when 'TRAVEL_여행' then 'mbr_url_global' when 'DATA_모바일/기기' then 'mbr_url_data' when 'OPEN_MEMBERSHIP_베이커리' then 'mbr_url_bakery' when 'VIP_PICK_#' then 'mbr_url_vippick' 
                            end as reco_type
        from comm.tmembership_reco_user_available_benefit
        where dt = '2020-09-01'
        -- and benefit_type in ('VIP_PICK', 'CULTURE', 'TRAVEL', 'DATA', 'OPEN_M

생성 결과는 다음과 같습니다.

In [4]:
%%bq

select *
from temp_1d.example_prediction_post_proc

BigQuery execution took 4 seconds.


,svc_mgmt_num,prod_id,prod_nm,impression_yn,score,percent_rk,reco_type,model,dt
0,015a611fc5f8fadbbdd38b227f91f16821dad1263d048f...,62,열린 베이커리 제휴점:::10% 할인,Y,0.557116,1,mbr_url_bakery,example_model,2020-09-01
1,00fc1d85bdb900c488a7690291e57721d93241d7a5b006...,62,열린 베이커리 제휴점:::10% 할인,Y,0.303755,1,mbr_url_bakery,example_model,2020-09-01
2,01087cb6a13c7f4515e5fd2cc6c8794981e29d89492a47...,62,열린 베이커리 제휴점:::10% 할인,Y,0.282224,1,mbr_url_bakery,example_model,2020-09-01
3,0109a7387a40c30e9c36aad76a0840d74c4b802a7290ac...,62,열린 베이커리 제휴점:::10% 할인,Y,0.775433,1,mbr_url_bakery,example_model,2020-09-01
4,010e7d6a4ca641bf23d222b92439793b23a4395e707f46...,62,열린 베이커리 제휴점:::10% 할인,Y,0.578562,1,mbr_url_bakery,example_model,2020-09-01
5,0160a18c97c7a98aa838cfb2d54acee1756b2d6ee65983...,62,열린 베이커리 제휴점:::10% 할인,Y,0.356918,1,mbr_url_bakery,example_model,2020-09-01
6,01b3d484d5c5791c2ec9910c9095f989a56fad99f2fe3d...,62,열린 베이커리 제휴점:::10% 할인,Y,0.423727,1,mbr_url_bakery,example_model,2020-09-01
7,022487a2c010e8183bac411416694240c0ebeb7a3deb09...,62,열린 베이커리 제휴점:::10% 할인,Y,0.565421,1,mbr_url_bakery,example_model,2020-09-01
8,013ef8073f74164064bf997fc5ff396be2acfbac18910d...,62,열린 베이커리 제휴점:::10% 할인,Y,0.780705,1,mbr_url_bakery,example_model,2020-09-01
9,01e76138e5b2becca7be71b10f5854e78c00e33cf18abc...,62,열린 베이커리 제휴점:::10% 할인,Y,0.387984,1,mbr_url_bakery,example_model,2020-09-01


가공한 데이터를 JSON 포맷으로 변환합니다. 

In [5]:
query = f"""
    SELECT  a.svc_mgmt_num
    ,       to_json_string (
                struct (
                    a.svc_mgmt_num as user_id
                ,   a.items as items
                )
            ) as results
    ,       a.reco_type
    ,       a.model
    ,       a.dt 
    FROM
    (
        SELECT  svc_mgmt_num
        ,       reco_type
        ,       model
        ,       dt
        ,       array_agg(item) as items
        FROM
        (
            SELECT  svc_mgmt_num
            ,       reco_type
            ,       model
            ,       dt
            ,       struct
                    (
                        prod_id as id
                    ,   prod_nm as name
                    ,   struct
                        (
                            cast(score as string) as score
                        ) as props
                    ,   reco_type as type
                    ) as item
            FROM    
            (
                select svc_mgmt_num, 
                       prod_id, 
                       prod_nm, 
                       if(a.rn <= 4, 'Y', 'N') as impression_yn,
                       score, 
                       percent_rk, 
                       reco_type, 
                       model, 
                       dt 
                from (
                    SELECT  *, 
                            row_number() over (partition by svc_mgmt_num ORDER BY score DESC) as rn
                    FROM    temp_1d.example_prediction_post_proc
                    WHERE   dt = PARSE_DATE('%Y-%m-%d', '2020-09-01')
                    ) a 
            ) temp
            where impression_yn = 'Y'
        ) temp
        GROUP BY    svc_mgmt_num
        ,           reco_type
        ,           model
        ,           dt 
    ) a
"""

bq_insert_overwrite(
    sql=query,
    destination="emart-datafabric.temp_1d.example_prediction_json"
)

query: 
    SELECT  a.svc_mgmt_num
    ,       to_json_string (
                struct (
                    a.svc_mgmt_num as user_id
                ,   a.items as items
                )
            ) as results
    ,       a.reco_type
    ,       a.model
    ,       a.dt 
    FROM
    (
        SELECT  svc_mgmt_num
        ,       reco_type
        ,       model
        ,       dt
        ,       array_agg(item) as items
        FROM
        (
            SELECT  svc_mgmt_num
            ,       reco_type
            ,       model
            ,       dt
            ,       struct
                    (
                        prod_id as id
                    ,   prod_nm as name
                    ,   struct
                        (
                            cast(score as string) as score
                        ) as props
                    ,   reco_type as type
                    ) as item
            FROM    
            (
                select svc_mgmt_num, 
             

JSON으로 변환된 데이터를 조회합니다. 여기까지 MLS에 서빙하기 위한 데이터를 만드는 과정이었습니다.  

In [6]:
%%bq

SELECT *
FROM temp_1d.example_prediction_json

BigQuery execution took 4 seconds.


,svc_mgmt_num,results,reco_type,model,dt
0,022487a2c010e8183bac411416694240c0ebeb7a3deb09...,"{""user_id"":""022487a2c010e8183bac411416694240c0...",mbr_url_bakery,example_model,2020-09-01
1,01fbf9f08059b951121c6ddf4e55132623d464ae939fc5...,"{""user_id"":""01fbf9f08059b951121c6ddf4e55132623...",mbr_url_bakery,example_model,2020-09-01
2,01e76138e5b2becca7be71b10f5854e78c00e33cf18abc...,"{""user_id"":""01e76138e5b2becca7be71b10f5854e78c...",mbr_url_bakery,example_model,2020-09-01
3,013ef8073f74164064bf997fc5ff396be2acfbac18910d...,"{""user_id"":""013ef8073f74164064bf997fc5ff396be2...",mbr_url_bakery,example_model,2020-09-01
4,00fc1d85bdb900c488a7690291e57721d93241d7a5b006...,"{""user_id"":""00fc1d85bdb900c488a7690291e57721d9...",mbr_url_bakery,example_model,2020-09-01
5,01b3d484d5c5791c2ec9910c9095f989a56fad99f2fe3d...,"{""user_id"":""01b3d484d5c5791c2ec9910c9095f989a5...",mbr_url_bakery,example_model,2020-09-01
6,015a611fc5f8fadbbdd38b227f91f16821dad1263d048f...,"{""user_id"":""015a611fc5f8fadbbdd38b227f91f16821...",mbr_url_bakery,example_model,2020-09-01
7,010e7d6a4ca641bf23d222b92439793b23a4395e707f46...,"{""user_id"":""010e7d6a4ca641bf23d222b92439793b23...",mbr_url_bakery,example_model,2020-09-01
8,0160a18c97c7a98aa838cfb2d54acee1756b2d6ee65983...,"{""user_id"":""0160a18c97c7a98aa838cfb2d54acee175...",mbr_url_bakery,example_model,2020-09-01
9,01087cb6a13c7f4515e5fd2cc6c8794981e29d89492a47...,"{""user_id"":""01087cb6a13c7f4515e5fd2cc6c8794981...",mbr_url_bakery,example_model,2020-09-01


## S3 업로드

JSON으로 가공된 데이터를 MLS에서 지정한 S3 경로에 저장해야 추천데이터를 서빙할 수 있습니다.  
위에서 생성한 temp_1d.example_prediction_json 테이블 데이터의 value 컬럼을 지정된 S3 경로에 text 파일로 전송하면 됩니다.  
파일 전송은 Spark을 이용합니다. 우선 temp_1d.example_prediction_json 테이블 데이터를 Spark DataFrame으로 변환합니다.  
results 컬럼의 데이터만 필요하므로 아래와 같이 쿼리하여 Spark DataFrame을 생성합니다.

In [7]:
from pydatafabric.gcp import bq_to_df

df = bq_to_df("SELECT results FROM temp_1d.example_prediction_json")

Spark DataFrame을 지정된 경로로 전송합니다. 여기서는 예제를 설명하고 있으므로 실제 S3 경로에 넣지 않고 HDFS에 전송하겠습니다.  
실제로는 아래 코드의 HDFS 경로 대신 S3 경로만 지정해주면 쉽게 전송이 가능합니다.

In [8]:
# S3로 전송하는 경우 아래 경로를 S3 경로로 바꿔주면 됩니다.
output_path = "/data/tmp/example_s3_data"

df.write.mode("overwrite").text(output_path)

mlops-sdk를 사용하여 추천 모델을 서빙하기 위해서 먼저 AI/ML 담당(정유선)에 문의하시면 됩니다.
mlops-sdk에 대한 자세한 내용은 <a href="https://rec.shinsegae.ai/swagger/index.html" target="_blank">mlops-sdk 문서</a>를 참고하시기 바랍니다.